Solution: https://youtu.be/EdTCHhhzoR8+8

In [1]:
import os

os.environ["OPENAI_API_KEY"] =
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
# from langchain.llms import OpenAI # this code has been deprecated since recording.
from langchain.chat_models import ChatOpenAI # this is the replacement 
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample 

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [4]:
from langchain.document_loaders import CSVLoader

# TODO: load reviews from tv-reviews.csv
loader=CSVLoader("tv-reviews.csv")
documents = loader.load()

Then, let's initialize our LLM

In [5]:
# TODO: initialize OpenAI object with your API key
llm = ChatOpenAI()

Now, let's setup our parser and a template  - 

**Note**  that since recording, the code to initialize the model has been updated to 

`llm = ChatOpenAI()`

In [9]:
class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review, starting from 0")
        
parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
# TODO: setup a template with partial and input variables
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Negatives", "type": "array"}}, "required": ["positives", "negatives"]}
```


Pick 3 sample reviews to classify - LLMs have a limited context window they can work with. In later exercises, we'll see how to deal with that differently

In [19]:
# TODO: pick 3 random reviews and save them into reviews_to_classify variable
prompt=PromptTemplate(
    template="{question}\n{format_instructions}\nContext:\n{context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}

)
question="""
    Review TVs provided in the context.
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs.
    If there are no positive or negative reviews, output an empty JSON array.
"""
reviews_to_classify = sample(documents, 3)
context="\n".join([review.page_content for review in reviews_to_classify])
query=prompt.format(context=context, question=question)
print(f"{query=}")
output=llm.predict(query)
print(f"{output=}")
result=parser.parse(output)
print(f"{result=}")


query='\n    Review TVs provided in the context.\n    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs.\n    If there are no positive or negative reviews, output an empty JSON array.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 0", "items": {"minim

    # generate textual prompt from the prompt template
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(context = context, question = question)

Finally, let's send our query to LLM and use the parser we setup to parse an output into a Python object

**NOTE**: Since recording the code to feed the query to the llm has been updated to

`llm.predict(query)`

In [20]:
# TODO: query LLM, then parse output into the result variable

print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))

Positives:
TV Name: VisionMax Ultra
Review Title: Immersive Audio Experience
Review Rating: 10
Review Text: The VisionMax Ultra delivers an immersive audio experience. The built-in speakers produce clear and well-balanced sound. The Dolby Atmos technology adds depth and dimension to the audio, making it feel like I'm in a theater. I'm thoroughly impressed!
TV Name: VisionMax Ultra
Review Title: Easy Installation Process
Review Rating: 9
Review Text: Setting up the VisionMax Ultra was a breeze. The instructions provided were clear and concise, making the installation process quick and hassle-free. The TV also offers various mounting options, allowing me to choose the perfect placement for my viewing pleasure.
TV Name: Imagix Pro
Review Title: Unmatched Clarity
Review Rating: 10
Review Text: I cannot express enough how impressed I am with the clarity of the Imagix Pro. Every detail is so sharp and lifelike, it's like I can reach out and touch the images on the screen. The colors are vibr